In [1]:
################################################################################################################################
#RECOMMENDATION ENGINE
#by Combination of User Similarity Clustering,Content Based Recommendation and Top Trending Recommendation
################################################################################################################################




import pandas as pd
import numpy as np
import math
from scipy.spatial.distance import cosine
from sklearn.cluster import KMeans
from sklearn.externals import joblib


################################################################################################################################
#User Affiliation Rating
################################################################################################################################

Ratings=pd.read_csv("data_set.csv",encoding='utf-8') #userId	movieId	rating	timestamp
merch_database=pd.read_csv("curated tags.csv",encoding='utf-8')
Data =pd.DataFrame()
MScore=Ratings[['merchant_id','is_offer','top90_flag','behav_type']]
MScore=MScore.drop_duplicates()
MScore =MScore.reset_index()
MScore =MScore[['merchant_id','is_offer','top90_flag','behav_type']]
MScore['gate1']=0
MScore['gate2']=0
MScore['gate3']=0
MScore['gate1'][MScore['is_offer'] ==1]=1
MScore['gate2'][MScore['top90_flag'] =='yes']=1
MScore['gate3'][MScore['behav_type'] =='taste']=1
MScore['merch_rating']=MScore['gate1']+MScore['gate2']+MScore['gate3']
MScore =MScore[['merchant_id','merch_rating']]
UScore =Ratings[['customer_id','merchant_id','user_transaction_amount','merch_avg_trasaction','merch_total_txn']]
UScoredataTF =UScore.groupby(['customer_id','merchant_id'],as_index =False,sort=False).count().rename(columns={'merch_total_txn':'user_txn_count'})[['customer_id','merchant_id','user_txn_count']]
TotMerTransac =UScore.groupby(['merchant_id'],as_index =False,sort=False).count().rename(columns={'customer_id':'tot_merch_txn'})[['merchant_id','tot_merch_txn']]
TxnScore =pd.merge(TotMerTransac,UScoredataTF,on='merchant_id',how='left',sort=False)
TxnScore['txn_rating'] =TxnScore['user_txn_count']/TxnScore['tot_merch_txn']
TxnScore['txn_rating'] =TxnScore['txn_rating']
TxnAmt =UScore.groupby(['customer_id'],as_index =False,sort= False).sum().rename(columns={'user_transaction_amount':'user_tot_amount'})[['customer_id','user_tot_amount']]
UserMerTxnAmt=UScore.groupby(['customer_id','merchant_id'],as_index=False,sort=False).sum().rename(columns={'user_transaction_amount':'user_merch_txnamount'})[['customer_id','merchant_id','user_merch_txnamount']]
TxnAmtScore =pd.merge(TxnAmt,UserMerTxnAmt,on='customer_id',how='left',sort=False)
TxnAmtScore['txn_amount_rating']=TxnAmtScore['user_merch_txnamount']/TxnAmtScore['user_tot_amount']
UScoreRating =pd.merge(TxnScore,TxnAmtScore,on=['customer_id','merchant_id'],how='left',sort=False)
UScoreRating =UScoreRating[['customer_id','merchant_id','txn_rating','txn_amount_rating']]
UScoreRating['gate1']=0

UScoreRating['gate1'][UScoreRating['txn_amount_rating'] < 0.25]=0.
UScoreRating['gate1'][(UScoreRating['txn_amount_rating']>=0.25) & (UScoreRating['txn_amount_rating']<0.50)]=0.25
UScoreRating['gate1'][(UScoreRating['txn_amount_rating']>=0.50) & (UScoreRating['txn_amount_rating']<0.75)]=0.50
UScoreRating['gate1'][(UScoreRating['txn_amount_rating']>=0.75)]=0.75
UScoreRating['gate1'][(UScoreRating['txn_amount_rating']==1)]=1

Final_Score =pd.merge(UScoreRating,MScore,on='merchant_id',how='left',sort=False)
Final_Score['cust_affltn_rating'] =Final_Score['gate1']+Final_Score['txn_rating']+Final_Score['merch_rating']
Final_Score=Final_Score[['customer_id','merchant_id','cust_affltn_rating']]

Final_Score =Final_Score.sort('cust_affltn_rating', ascending=False)
Final_Score =Final_Score.rename(columns={'merchant_id':'merch_id'})[['merch_id','cust_affltn_rating']]
Final_Score['count']=0
Final_Score1 =Final_Score.groupby(['merch_id'],as_index =False,sort=False).count().rename(columns={'count':'Count'})[['merch_id','Count']]
Final_Score = Final_Score.groupby(['merch_id'],as_index =False,sort=False).sum().rename(columns={'cust_affltn_rating':'tot_cust_affltn_rating'})[['merch_id','tot_cust_affltn_rating']]

Final_Score = pd.merge(Final_Score,Final_Score1,on='merch_id',how='left',sort=False)
Final_Score['Rating']=Final_Score['tot_cust_affltn_rating']/Final_Score['Count']
Final_Score =Final_Score[['merch_id','Rating']]
Final_Score =pd.merge(merch_database,Final_Score,on='merch_id',how='left',sort=False)
Final_Score =Final_Score[['merch_id','merch_name','Rating']]
Final_Score['Rating'] =Final_Score['Rating'].fillna(0)


################################################################################################################################
#User-User Similarity Matrix
################################################################################################################################

Tags=pd.read_csv("tags_elist.csv",encoding='utf-8')
UTF = Tags.groupby(['customer_id','tags'],as_index =False,sort =False).count().rename(columns ={'merch_id':'tag_count_UTF'})[['customer_id','tags','tag_count_UTF']]
Users_Distinct =Tags[['customer_id','tags']].drop_duplicates()

UDF =Users_Distinct.groupby(['tags'],as_index =False,sort=False).count().rename(columns={'customer_id':'tag_count_UDF'})[['tags','tag_count_UDF']]

a =math.log10(len(np.unique(Tags['customer_id'])))
UDF['UDF']=a-np.log10(UDF['tag_count_UDF'])
UTF =pd.merge(UTF,UDF,on ='tags',how ='left',sort =False)
UTF['UTF-UDF']=UTF['tag_count_UTF']*UTF['UDF']
Vect_len=UTF[['customer_id','UTF-UDF']]
Vect_len['UTF-UDF-Sq'] =Vect_len['UTF-UDF']**2
Vect_len = Vect_len.groupby('customer_id',as_index =False,sort =False).sum().rename(columns ={'UTF-UDF-Sq':'UTF-UDF-Sq-sum'})[['customer_id','UTF-UDF-Sq-sum']]
Vect_len['vect_len'] = np.sqrt(Vect_len[['UTF-UDF-Sq-sum']].sum(axis=1))
UTF =pd.merge(UTF,Vect_len,on='customer_id',how='left',sort =False)
UTF['UserTAG_WT']=UTF['UTF-UDF']/UTF['vect_len']

Sample =UTF['customer_id'][:1000].drop_duplicates()
sample_count =len(Sample)
data_ibs = pd.DataFrame(index=Sample,columns=Sample)
for i in range(0,len(data_ibs.columns)) :
    for j in range(0,len(data_ibs.columns)) :
     try:
      data_ibs.ix[i,j] = 1-cosine(UTF['UserTAG_WT'].ix[i-1:i],UTF['UserTAG_WT'].ix[j-1:j])
     except:
      pass
data_ibs = data_ibs.drop('UID6203', 1)
data_ibs =data_ibs.drop(['UID6203'])
num_clusters = 25
km = KMeans(n_clusters=num_clusters)
%time km.fit(data_ibs)
clusters = km.labels_.tolist()
joblib.dump(km,  'doc_cluster.pkl')
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()


Ucluster_Dict = {'customer_id':Sample[:sample_count-1],'clusters': clusters[:sample_count-1]}
Ucluster_Data = pd.DataFrame(Ucluster_Dict, columns = ['customer_id','clusters'])
Ucluster_Data = Ucluster_Data.sort(['clusters'], ascending=True)


################################################################################################################################
#Calculating Tf IDf Scores for finding User Tag Preference
################################################################################################################################



Tags=pd.read_csv("tags_elist.csv",encoding='utf-8')#movieId	userId	tag	timestamp
#Ratings=pd.read_csv("ratings_temp.csv",encoding='utf-8') #userId	movieId	rating	timestamp
TF = Tags.groupby(['merch_id','tags'],as_index =False,sort =False).count().rename(columns ={'customer_id':'tag_count_TF'})[['merch_id','tags','tag_count_TF']]
TF.is_copy = False
TF=pd.merge(TF,Tags,on=['tags','merch_id'],how='left',sort=False)
TAGS_Distinct =Tags[['merch_id','tags']].drop_duplicates()
DF =TAGS_Distinct.groupby(['tags'],as_index =False,sort=False).count().rename(columns={'merch_id':'tag_count_DF'})[['tags','tag_count_DF']]
a =math.log10(len(np.unique(Tags['merch_id'])))
DF['IDF']=a-np.log10(DF['tag_count_DF'])
TF =pd.merge(TF,DF,on ='tags',how ='left',sort =False)
TF['TF-IDF']=TF['tag_count_TF']*TF['IDF']
Vect_len=TF[['merch_id','TF-IDF']]
Vect_len['TF-IDF-Sq'] =Vect_len['TF-IDF']**2
Vect_len = Vect_len.groupby('merch_id',as_index =False,sort =False).sum().rename(columns ={'TF-IDF-Sq':'TF-IDF-Sq-sum'})[['merch_id','TF-IDF-Sq-sum']]
Vect_len['vect_len'] = np.sqrt(Vect_len[['TF-IDF-Sq-sum']].sum(axis=1))
TF =pd.merge(TF,Vect_len,on='merch_id',how='left',sort =False)
TF['TAG_WT']=TF['TF-IDF']/TF['vect_len']



user_tag_pref =pd.DataFrame()
tot_clusters =len(np.unique(clusters))
for cluster in range(tot_clusters):
    print ('\n','cluster: ',cluster,'out of: ',tot_clusters)
    Cluster_data =Ucluster_Data[Ucluster_Data['clusters']==cluster]
    distinct_cust_cluster =np.unique(Cluster_data['customer_id'])
#     print (distinct_cust_cluster)
    i=1
    for customer in distinct_cust_cluster:
        print ("user: ", i , "out of: ", len(distinct_cust_cluster))
        user_data = TF[TF['customer_id']==customer]
        user_data1 = user_data.groupby(['tags'], as_index = False, sort = False).sum().rename(columns = {'TAG_WT':'tag_pref'})[['tags','tag_pref']]
        user_data1['customer']=customer
        user_data1['clusters']=cluster
        user_tag_pref =user_tag_pref.append(user_data1,ignore_index =True)
        print (user_data1,'\n')
        i=i+1

C:\Users\SUNDARESAN\Anaconda3\lib\site-packages\ipykernel\__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\SUNDARESAN\Anaconda3\lib\site-packages\ipykernel\__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\SUNDARESAN\Anaconda3\lib\site-packages\ipykernel\__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\SUNDARESAN\Anaconda3\lib\site-packages\ipykernel\__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy

Wall time: 46.9 ms


C:\Users\SUNDARESAN\Anaconda3\lib\site-packages\ipykernel\__main__.py:117: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\SUNDARESAN\Anaconda3\lib\site-packages\ipykernel\__main__.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



 cluster:  0 out of:  25
user:  1 out of:  28
                         tags  tag_pref customer  clusters
0            money withdrawal  0.011633   UID124         0
1                  mobile app  0.334104   UID124         0
2              online payment  0.135484   UID124         0
3             online services  0.072984   UID124         0
4             online recharge  0.121477   UID124         0
5               mobile wallet  0.083526   UID124         0
6                bill payment  0.135484   UID124         0
7   Internet service provider  0.155395   UID124         0
8          broadband services  0.155395   UID124         0
9                         b2c  0.116356   UID124         0
10                        b2b  0.103747   UID124         0
11              trusted brand  0.061848   UID124         0
12    mobile network operator  0.140127   UID124         0
13              data services  0.140127   UID124         0
14         financial services  0.083339   UID124         0
15       

In [2]:
################################################################################################################################
#Content Based Recommendation
################################################################################################################################

tag_merge_all=pd.DataFrame()
for cluster in range(tot_clusters):
    print ('\n','cluster: ',cluster,'out of: ',tot_clusters)
    Cluster_data =user_tag_pref[user_tag_pref['clusters']==cluster]
    distinct_cust_cluster =np.unique(Cluster_data['customer'])
    i=1
    for customer in distinct_cust_cluster:
        print ('\t','customer: ',i,'out of: ',len(distinct_cust_cluster))
        user_tag_pref_all= user_tag_pref[user_tag_pref['customer']==customer]
        distinct_merchants = np.unique(TF['merch_id'])
        j=1
        for merchant in distinct_merchants:
            if j%10!=0:
                #print ("merchant: ", j," merchant_id",merchant, "out of: ", len(distinct_merchants) , "with user: ", i , "out of: ", len(distinct_cust_cluster), " in cluster ",cluster)
                TFMerchant =TF[TF['merch_id']==merchant]
                tag_merge =pd.merge(TFMerchant,user_tag_pref_all,on='tags',how='left',sort = False)
                tag_merge['tag_pref'] =tag_merge['tag_pref'].fillna(0)
                tag_merge['tag_value'] =tag_merge['TAG_WT']*tag_merge['tag_pref']
                tag_merge['customer']=customer
                TAG_WT_val=np.sqrt(np.sum(np.square(tag_merge['TAG_WT']), axis=0))
                tag_pref_val=np.sqrt(np.sum(np.square(user_tag_pref_all['tag_pref']), axis=0))
                tag_merge_final = tag_merge.groupby(['customer','merch_id'])[['tag_value']].sum().rename(columns = {'tag_value': 'Rating'}).reset_index()
                tag_merge_final['Rating']=tag_merge_final['Rating']/(TAG_WT_val*tag_pref_val)
                tag_merge_all = tag_merge_all.append(tag_merge_final, ignore_index=True)
                #print(tag_merge_final.head())
                j=j+1
        i=i+1


merch_database=pd.read_csv("curated tags.csv",encoding='utf-8')
dis_users =np.unique(tag_merge_all['customer'])
for user in dis_users:
    print ("Hi user:",user,"Here are some of your featured recommendations :")
    tag_merge_all2 =tag_merge_all[tag_merge_all['customer']==user]
    tag_merge_all2 = tag_merge_all2.sort_index(ascending= False,by=['Rating']).reset_index(drop =True)
    tag_merge_all2 = pd.merge(tag_merge_all2,merch_database,on='merch_id',how='left',sort=False)[['customer','merch_id','merch_name','category']]
    distinct_category= np.unique(tag_merge_all2['category'])
    for category in distinct_category:
        print ("Recommendation under ",category,"category :")
        recommendations =tag_merge_all2[tag_merge_all2['category']==category]
        print (recommendations)


 cluster:  0 out of:  25
	 customer:  1 out of:  28
	 customer:  2 out of:  28
	 customer:  3 out of:  28
	 customer:  4 out of:  28
	 customer:  5 out of:  28
	 customer:  6 out of:  28
	 customer:  7 out of:  28
	 customer:  8 out of:  28
	 customer:  9 out of:  28
	 customer:  10 out of:  28
	 customer:  11 out of:  28
	 customer:  12 out of:  28
	 customer:  13 out of:  28
	 customer:  14 out of:  28
	 customer:  15 out of:  28
	 customer:  16 out of:  28
	 customer:  17 out of:  28
	 customer:  18 out of:  28
	 customer:  19 out of:  28
	 customer:  20 out of:  28
	 customer:  21 out of:  28
	 customer:  22 out of:  28
	 customer:  23 out of:  28
	 customer:  24 out of:  28
	 customer:  25 out of:  28
	 customer:  26 out of:  28
	 customer:  27 out of:  28
	 customer:  28 out of:  28

 cluster:  1 out of:  25
	 customer:  1 out of:  12
	 customer:  2 out of:  12
	 customer:  3 out of:  12
	 customer:  4 out of:  12
	 customer:  5 out of:  12
	 customer:  6 out of:  12
	 customer:

C:\Users\SUNDARESAN\Anaconda3\lib\site-packages\ipykernel\__main__.py:39: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


  customer merch_id                 merch_name category
7  UID2081  MID1179  G R THANGA MALIGAI JEWELL   retail
Recommendation under  utilities category :
  customer   merch_id                 merch_name   category
0  UID2081  MID111362                 ICICI BANK  utilities
1  UID2081    MID1136                  AXIS BANK  utilities
2  UID2081    MID1213                     BESCOM  utilities
3  UID2081  MID102445       BANGALORE ONE BESCOM  utilities
4  UID2081    MID1198  MAX LIFE INSURANCE CO LTD  utilities
5  UID2081    MID1197     AIRTEL MOBILE AND SIMS  utilities
6  UID2081    MID1210                     MCAFEE  utilities
Hi user: UID2150 Here are some of your featured recommendations :
Recommendation under  miscellaneous category :
  customer merch_id    merch_name       category
8  UID2150    MID10  CESC LIMITED  miscellaneous
Recommendation under  retail category :
  customer merch_id                 merch_name category
7  UID2150  MID1179  G R THANGA MALIGAI JEWELL   retail
Re

In [ ]:
###############################################################################################################################
#Combination of Content Based Recommendation with Cross Categories and Top Trending Recommendation
###############################################################################################################################
big_box=pd.DataFrame()

tot_clusters =len(np.unique(clusters))
for cluster in range(tot_clusters):#tot_clusters
    print ('\n','cluster: ',cluster,'out of: ',tot_clusters)
    Ucluster =Ucluster_Data[Ucluster_Data['clusters']==cluster]
    distinct_user_ids= np.unique(Ucluster['customer_id'])
    distinct_movie_ids=np.unique(Tags['merch_id'])
    for user in distinct_user_ids:
        user_mcs = Tags[Tags['customer_id']==user][['customer_id','merch_id']].drop_duplicates()
        user_mcs['clusters']=cluster
        print (user_mcs)
        mcs =np.unique(user_mcs['merch_id'])
        big_box =big_box.append(user_mcs,ignore_index =False)


for user in dis_users:
    df_box=pd.DataFrame()
#     big_box=pd.DataFrame()
    counter=0
    for cluster in range(tot_clusters):#tot_clusters
    #    print ('\n','cluster: ',cluster,'out of: ',tot_clusters)
        Ucluster =Ucluster_Data[Ucluster_Data['clusters']==cluster]
        distinct_user_ids= np.unique(Ucluster['customer_id'])
        distinct_merch_ids=np.unique(Tags['merch_id'])
        usercheck =Ucluster[Ucluster['customer_id']==user]
        usercheck =Ucluster[Ucluster['customer_id']==user]
        usercheck =usercheck['customer_id'].tolist()
        if len(usercheck) >0:
            user_mcs = Tags[Tags['customer_id']==user][['customer_id','merch_id']].drop_duplicates()
            #mcs = [tuple(x) for x in user_mcs['merch_id'].values]
            user_rec =big_box[big_box['clusters']==cluster]
            user_mcrec =user_rec[user_rec['customer_id'] !=user]
            print ("Top trending recommendations for you",user)
    #         print  (user_mcrec[['merch_id']].head(10))
            df =pd.merge(Final_Score,user_mcrec,on='merch_id',how='left',sort=False)
            df['customer_id'] =df['customer_id'].fillna(0)
            df = df[df['customer_id']!=0][['merch_id','merch_name']]#,'Rating']] 
            df_box =df_box.append(df,ignore_index =False)
            df_box =df_box.drop_duplicates()
            #df_box =df_box.sort_index(ascending= False,by=['Rating']).reset_index(drop =True)
            print  (df_box)


        if counter ==0:
            print ("Hi user:",user,"Here are some of your featured recommendations :")
            tag_merge_all2 =tag_merge_all[tag_merge_all['customer']==user]
            tag_merge_all2 = tag_merge_all2.sort_index(ascending= False,by=['Rating']).reset_index(drop =True)
            tag_merge_all2 = pd.merge(tag_merge_all2,merch_database,on='merch_id',how='left',sort=False)[['customer','merch_id','merch_name','category']]
            distinct_category= np.unique(tag_merge_all2['category'])
            for category in distinct_category:
                print ("Recommendation under ",category,"category :")
                recommendations =tag_merge_all2[tag_merge_all2['category']==category]
                print (recommendations)
            counter =1


 cluster:  0 out of:  25
       customer_id  merch_id  clusters
4191        UID124  MID96079         0
394907      UID124   MID2502         0
581483      UID124   MID7147         0
       customer_id  merch_id  clusters
4877       UID1392  MID96079         0
394727     UID1392   MID2502         0
581076     UID1392   MID7147         0
       customer_id merch_id  clusters
1191       UID1587  MID5377         0
400239     UID1587   MID338         0
       customer_id merch_id  clusters
2541       UID1901  MID5377         0
414283     UID1901   MID338         0
       customer_id merch_id  clusters
1938       UID1927  MID5377         0
413695     UID1927   MID338         0
     customer_id merch_id  clusters
1371     UID2046  MID5377         0
       customer_id merch_id  clusters
3243       UID2150  MID5377         0
400351     UID2150   MID338         0
       customer_id merch_id  clusters
2415       UID2176  MID5377         0
416355     UID2176   MID338         0
       customer_id m

C:\Users\SUNDARESAN\Anaconda3\lib\site-packages\ipykernel\__main__.py:51: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


Top trending recommendations for you UID1115
    merch_id              merch_name
7    MID2502           ATM CASH WDRL
54  MID96079        FASTRECHARGE.COM
65   MID5377             GODADDY.COM
76   MID7147  HATHWAY CABLE & DATACO
80    MID338           HDFC BANK LTD
Top trending recommendations for you UID124
    merch_id              merch_name
7    MID2502           ATM CASH WDRL
58  MID96079        FASTRECHARGE.COM
71   MID5377             GODADDY.COM
90   MID7147  HATHWAY CABLE & DATACO
97    MID338           HDFC BANK LTD
Hi user: UID124 Here are some of your featured recommendations :
Recommendation under  miscellaneous category :
  customer merch_id    merch_name       category
8   UID124    MID10  CESC LIMITED  miscellaneous
Recommendation under  retail category :
  customer merch_id                 merch_name category
7   UID124  MID1179  G R THANGA MALIGAI JEWELL   retail
Recommendation under  utilities category :
  customer   merch_id                 merch_name   category
0 

In [8]:
big_box

""


In [ ]:
################################################################################################################################
#Top Trending Recommendation for each cluster,rated by calculated user affiliation score at the starting 
################################################################################################################################
df_box=pd.DataFrame()
for cluster in range(tot_clusters):#tot_clusters
    print ('\n','cluster: ',cluster,'out of: ',tot_clusters)
    Ucluster =Ucluster_Data[Ucluster_Data['clusters']==cluster]
    distinct_user_ids= np.unique(Ucluster['customer_id'])
    distinct_merch_ids=np.unique(Tags['merch_id'])
    usercheck =Ucluster[Ucluster['customer_id']==user]
    usercheck =Ucluster[Ucluster['customer_id']==user]
    usercheck =usercheck['customer_id'].tolist()
    if len(usercheck) >0:
        user_mcs = Tags[Tags['customer_id']==user][['customer_id','merch_id']].drop_duplicates()
        #mcs = [tuple(x) for x in user_mcs['merch_id'].values]
        user_rec =big_box[big_box['clusters']==cluster]
        user_mcrec =user_rec[user_rec['customer_id'] !=user]
        print ("Top trending recommendations for you",user)
#         print  (user_mcrec[['merch_id']].head(10))
        df =pd.merge(Final_Score,user_mcrec,on='merch_id',how='left',sort=False)
        df['customer_id'] =df['customer_id'].fillna(0)
        df = df[df['customer_id']!=0][['merch_id','merch_name']]#,'Rating']] 
        df_box =df_box.append(df,ignore_index =False)
        df_box =df_box.drop_duplicates()
        #df_box =df_box.sort_index(ascending= False,by=['Rating']).reset_index(drop =True)
        print  (df_box)